# 서울시 생활인구
## 서울 생활인구 현황 (2022.09.28. 기준)
### url: https://data.seoul.go.kr/dataVisual/seoul/seoulLivingPopulation.do

1. 집계구 단위 서울 생활인구(내국인)
    - url: https://data.seoul.go.kr/dataList/OA-14979/F/1/datasetView.do
    - 설명: 서울시가 보유한 공공데이터와 통신데이터로 측정한 특정시점에 서울의 특정 지역에 존재하는 인구 중 내국인

<!-- <br> -->


2. 집계구 단위 서울 생활인구(장기체류 외국인)
    - url: https://data.seoul.go.kr/dataList/OA-14978/F/1/datasetView.do
    - 설명:서울시가 보유한 공공데이터와 통신데이터로 측정한 특정시점에 서울의 특정 지역에 존재하는 인구 중 장기체류 외국인

<!-- <br> -->


3. 집계구 단위 서울 생활인구(단기체류 외국인)
    - url: https://data.seoul.go.kr/dataList/OA-14980/F/1/datasetView.do
    - 설명:서울시가 보유한 공공데이터와 통신데이터로 측정한 특정시점에 서울의 특정 지역에 존재하는 인구 중 단기체류 외국인

<!-- <br> -->



※ 개인정보 비 식별화를 위하여 ‘3명’ 이하인 경우 “ * ” 처리


In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
from tqdm.auto import tqdm, trange
from time import sleep

## 1. 집계구 단위 서울 생활인구(내국인)
    - a)2022년 01월~08월
    - b)2019년, 2020년, 2021년
    - 2019년 10월 15-27일 데이터 없음
## 2. 집계구 단위 서울 생활인구(장기체류 외국인)
    - a)2022년 01월~08월
    - b)2019년, 2020년, 2021년
## 3. 집계구 단위 서울 생활인구(단기체류 외국인)
    - a)2022년 01월~08월
    - b)2019년, 2020년, 2021년

In [3]:
# 폴더 안 파일명 리스트 가져오기
local_people_2019_list = []
local_people_2020_list = []
local_people_2021_list = []

long_foreigner_list = []
temp_foreinger_list= []
pbar = tqdm(os.listdir(r"./data/집계동 단위 서울 생활인구/local_people/2019"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        local_people_2019_list.append(file)
        
pbar = tqdm(os.listdir(r"./data/집계동 단위 서울 생활인구/local_people/2020"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        local_people_2020_list.append(file)
        
pbar = tqdm(os.listdir(r"./data/집계동 단위 서울 생활인구/local_people/2021"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        local_people_2021_list.append(file)

pbar = tqdm(os.listdir(r"./data/집계동 단위 서울 생활인구/long_foreigner/"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        long_foreigner_list.append(file)
 
pbar = tqdm(os.listdir(r"./data/집계동 단위 서울 생활인구/temp_foreigner/"))
for file in pbar:
    if file.endswith(".csv"):
        pbar.set_description(file)
        temp_foreinger_list.append(file)

print("local_people_2019_list >>", len(local_people_2019_list))
print("local_people_2020_list >>", len(local_people_2020_list))
print("local_people_2021_list >>", len(local_people_2021_list))

print("long_foreigner_list >>", long_foreigner_list)
print("temp_foreinger_list >>", temp_foreinger_list)

  0%|          | 0/352 [00:00<?, ?it/s]

  0%|          | 0/366 [00:00<?, ?it/s]

  0%|          | 0/365 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

local_people_2019_list >> 352
local_people_2020_list >> 366
local_people_2021_list >> 365
long_foreigner_list >> ['LONG_FOREIGNER_20220924.csv']
temp_foreinger_list >> ['TEMP_FOREIGNER_20220924.csv']


In [ ]:
def data_process(file_list, path):
    df_list = []
    for file in pbar:
        pbar.set_description(file)
        try:df_temp = pd.read_csv(path + file, encoding="euc-kr", na_values="*")
        except: df_temp = pd.read_csv(path + file, encoding="utf-8", na_values="*")
        df_list.append(df_temp)
        

In [4]:
# csv concat
def concat_files(file_list, path):
    df_list = []
    pbar = tqdm(file_list)
    for file in pbar:
        pbar.set_description(file)
        try:df_temp = pd.read_csv(path + file, encoding="euc-kr", na_values="*")
        except: df_temp = pd.read_csv(path + file, encoding="utf-8", na_values="*")
        df_list.append(df_temp)
    
    df = pd.concat(df_list)
    return df 

In [ ]:
local_people_2019_df = concat_files(local_people_2019_list, "./data/집계동 단위 서울 생활인구/local_people/2019/")
local_people_2020_df = concat_files(local_people_2020_list, "./data/집계동 단위 서울 생활인구/local_people/2020/")
local_people_2021_df = concat_files(local_people_2021_list, "./data/집계동 단위 서울 생활인구/local_people/2021/")


long_foreigner_df = concat_files(long_foreigner_list, "./data/집계동 단위 서울 생활인구/long_foreigner/")
temp_foreinger_df = concat_files(temp_foreinger_list, "./data/집계동 단위 서울 생활인구/temp_foreigner/")
print("local_people_df.shape >>", local_people_df.shape)
print("long_foreigner_df.shape >>", long_foreigner_df.shape)
print("temp_foreinger_df.shape >>", temp_foreinger_df.shape)

  0%|          | 0/352 [00:00<?, ?it/s]

In [ ]:
# 결측값 0으로 대체
local_people_df.fillna(0, inplace=True)
local_people_df.info()

In [ ]:
cols = local_people_df.columns
cols = ['남자10세부터14세생활인구수', '남자15세부터19세생활인구수', '남자20세부터24세생활인구수',
       '남자25세부터29세생활인구수', '남자30세부터34세생활인구수', '남자35세부터39세생활인구수',
       '남자40세부터44세생활인구수', '남자45세부터49세생활인구수', '남자50세부터54세생활인구수',
       '남자55세부터59세생활인구수', '남자60세부터64세생활인구수', '남자65세부터69세생활인구수', '남자70세이상생활인구수',
       '여자0세부터9세생활인구수', '여자10세부터14세생활인구수', '여자15세부터19세생활인구수',
       '여자20세부터24세생활인구수', '여자25세부터29세생활인구수', '여자30세부터34세생활인구수',
       '여자35세부터39세생활인구수', '여자40세부터44세생활인구수', '여자45세부터49세생활인구수',
       '여자50세부터54세생활인구수', '여자55세부터59세생활인구수', '여자60세부터64세생활인구수',
       '여자65세부터69세생활인구수', '여자70세이상생활인구수']
cols

In [ ]:
for col in tqdm(cols):
    if col

In [ ]:
local_people_df['10대 남성'] = local_people_df.남자10세부터14세생활인구수 + local_people_df.남자15세부터19세생활인구수
local_people_df['20대 남성'] = 